In [57]:
import os
import glob

from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize

from nltk import download

import pyemd as emd

from gensim import models
import gensim

import pandas as pd
import numpy as np


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yifeining/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [217]:
download('punkt')
download('stopwords')

[nltk_data] Downloading package punkt to /Users/yifeining/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yifeining/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 1. simple sentence trials

In [50]:
from gensim.models import KeyedVectors

if not os.path.exists('data/GoogleNews-vectors-negative300.bin'):
    raise ValueError("SKIP: You need to download the google news model")

model = KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)
# auto-normalize vecs; keep the length uniform


In [179]:
for_test = 'I sell drug to poor people and make money from them'
agst_test = 'cigar is bad for people\'s health, and you should quit smoking'
temp = 'I love cigar-flavored icecream'

temp = temp.lower().split()
for_test = for_test.lower().split()
agst_test = agst_test.lower().split()

In [180]:
for_test, agst_test

(['i',
  'sell',
  'drug',
  'to',
  'poor',
  'people',
  'and',
  'make',
  'money',
  'from',
  'them'],
 ['cigar',
  'is',
  'bad',
  'for',
  "people's",
  'health,',
  'and',
  'you',
  'should',
  'quit',
  'smoking'])

In [181]:
model.wmdistance(for_test, agst_test) # can't distinguish sides

1.1868173578497592

# 2. Word Mover Distance w/ google word emmbedding
    - GoogleNews-vectors-negative300.bin

In [111]:
def word_mover_distance(sentence_1, sentence_2, normalize = True):
    if not normalize:
        model.init_sims(replace=False)
    
    return model.wmdistance(sentence_1, sentence_2)

In [174]:
def load_script(path):
    '''
    take one csv file path
    return for/agst string corpus
    '''
#     for_corpus = ''
#     agst_corpus = ''
#     for filepath in glob.glob(data_path):

    if not os.path.exists(path):
        os.rename(path, path + '.csv')
        path = path + '.csv'
    
    df = pd.read_csv(path)
    for_script = ' '.join(list(df.loc[df.side == 'for', 'script']))
    agst_script = ' '.join(list(df.loc[df.side == 'against', 'script']))

#         for_corpus += for_scripts + ' '
#         agst_corpus += agst_scripts + ' '

    return for_script, agst_script


# def load_point_from_meta(path):
#     if path[-3:] != 'csv':
#         os.rename(path, path + '.csv')
#         path = path + '.csv'
    
#     df = pd.read_csv(path)
# #     print(df)
# #     for_side = df.loc[:, 'For The Motion']
# #     agst_side = df.loc[:, 'Against The Motion']
    
#     return df


# loading meta data w/ main pts for each side
    - noticed null entries; needs to be filtered

In [230]:
meta_path =  'DebateStar/Meta Data/metadata_appended_main_points.csv'
meta_raw = pd.read_csv(meta_path)
meta = meta_raw.loc[:, ['id', 'For_Main_Points', 'against_Main_Points']]\
        .rename({'against_Main_Points': 'against_mp',
                 'For_Main_Points': 'for_mp'}, axis = 1)
print(meta.shape)
meta.head()

(175, 3)


,id,for_mp,against_mp
0,d20191112,['By promoting market competition and rewardin...,['Capitalism serves the interests of large cor...
1,d20191029,"[""We're in the midst of a DNA revolution: Whil...","['While DNA is important, factors like familia..."
2,d20191022,['European regulators have declared war on Ame...,['Brussels isn’t waging war on Silicon Valley....
3,d20190917,['The United States government should follow t...,['Individuals should have the freedom to choos...
4,d20190912,NaN,NaN


# loading results data w/ winners 
    - noticed a high proportion of undecided; needs to be filtered

In [192]:
live_path = 'DebateStar/results_data/final_live.csv'
online_path = 'DebateStar/results_data/final_online.csv'

live_res = pd.read_csv(live_path).loc[:, ['id', 'winner']].rename({'winner': 'live_winner'}, axis = 1)
online_res = pd.read_csv(online_path).loc[:, ['id', 'winner']].rename({'winner': 'online_winner'}, axis = 1)

# need to be filtered
print('proportion of undecided in live: ', np.mean(live_res.live_winner == 'undecided'))
print('proportion of undecided in online : ', np.mean(online_res.online_winner == 'undecided'))
print(live_res.shape, online_res.shape)

proportion of undecided in live:  0.1242603550295858
proportion of undecided in online :  0.25443786982248523
(169, 2) (169, 2)


In [212]:
live_res = live_res.loc[live_res.live_winner != 'undecided', :]
online_res = online_res.loc[online_res.online_winner != 'undecided', :]

In [213]:
online_res.head()

,id,online_winner
0,d20191112,against
1,d20191029,for
2,d20191022,against
3,d20190917,against
4,d20190802,against


# merging results data w/ meta data
    - null entries
    - undecided votes for online
    - undecided votes for live

In [223]:
import ast

In [277]:
merged = meta.merge(live_res, how = 'inner', on = 'id').merge(online_res, on = 'id').drop_duplicates()

merged = merged.dropna()
merged.head()

,id,for_mp,against_mp,live_winner,online_winner
0,d20191112,['By promoting market competition and rewardin...,['Capitalism serves the interests of large cor...,against,against
1,d20191029,"[""We're in the midst of a DNA revolution: Whil...","['While DNA is important, factors like familia...",against,for
2,d20191022,['European regulators have declared war on Ame...,['Brussels isn’t waging war on Silicon Valley....,against,against
3,d20190917,['The United States government should follow t...,['Individuals should have the freedom to choos...,against,against
4,d20190802,['The prospect of China becoming an open and l...,['The United States and China are great compet...,against,against


# loading scripts for each id

In [297]:
stop_words = stopwords.words('english')

def clean_text(text):
    text = text.lower()
    text = word_tokenize(text)  # Split into words.
    text = [w for w in text if (not w in stop_words) and (w.isalpha())]
    return text


def padding_vec(vec):
    MAX_LEN = 6
    
    assert len(vec) <= 6, 'script has more than 6 pts'
    padding = MAX_LEN - len(vec)
    return vec + [0] * padding

def for_win(for_, agt_):
    
    # if vec1 is more remote vec2 wins
    out = 'against' if (np.sum(np.array(for_)) > np.sum(np.array(agt_))) else 'for'
    return out


padding_vec([1])

[1, 0, 0, 0, 0, 0]

In [279]:
### general cleaning

# convert mp to list obj
merged.loc[:, 'for_mp'] = merged.for_mp.apply(lambda x: [clean_text(s) for s in ast.literal_eval(x)])
merged.loc[:, 'against_mp'] = merged.against_mp.apply(lambda x: [clean_text(s) for s in ast.literal_eval(x)])

merged = merged.set_index('id')
merged.head()

,for_mp,against_mp,live_winner,online_winner
id,,,,
d20191112,"[[promoting, market, competition, rewarding, i...","[[capitalism, serves, interests, large, corpor...",against,against
d20191029,"[[midst, dna, revolution, personal, genetic, i...","[[dna, important, factors, like, familial, dyn...",against,for
d20191022,"[[european, regulators, declared, war, america...","[[brussels, waging, war, silicon, valley, inst...",against,against
d20190917,"[[united, states, government, follow, lead, na...","[[individuals, freedom, choose, manage, health...",against,against
d20190802,"[[prospect, china, becoming, open, liberal, st...","[[united, states, china, great, competitors, e...",against,against
...,...,...,...,...
d20070918,"[[democratization, middle, east, result, regio...","[[america, maintain, image, beacon, democracy,...",for,for
d20070207,"[[separation, church, state, heralded, corners...","[[attempt, limit, individual, ability, employ,...",for,for
d20061213,"[[hollywood, films, portraying, americans, vio...","[[fueled, unpopular, american, policies, pract...",against,against


In [299]:
pred = []
for idx in merged.index:
    # get main points
    for_mp = merged.loc[idx, 'for_mp']
    against_mp = merged.loc[idx, 'against_mp']
#     print(for_mp)
#     print(against_mp)

    # get script of both side
    path = 'DebateStar/For Against Scripts/for_against_scripts_bp_free_{}.csv'.format(idx)
    for_script, against_script = load_script(path)
    
    # cleaning script
    for_script, against_script = clean_text(for_script), clean_text(against_script)
    
    # cleaning main pts [checked; refer to previous cells]
    
    # project scripts onto main points
    for_sim = [word_mover_distance(for_script, i) for i in for_mp]
    against_sim = [word_mover_distance(against_script, i) for i in for_mp]

    diff = for_win(padding_vec(for_sim), padding_vec(against_sim))
    
    pred.append(diff)
    
    
    

In [305]:
pred[:10]

['against',
 'for',
 'against',
 'for',
 'for',
 'against',
 'for',
 'against',
 'for',
 'against']

In [307]:

print(np.mean(merged.live_winner.values == pred))

0.5877192982456141


In [308]:
print(np.mean(merged.online_winner.values == pred))

0.5350877192982456


# 2.WmdSimilarity

In [6]:
# def clean_text(text):
#     text = text.lower()
#     text = word_tokenize(text)  # Split into words.
#     text = [w for w in text if (not w in stop_words) and (w.isalpha())]
#     return text


In [309]:
# df_p = 'for_against_scripts_d20100413.csv'
# main_p = 'main_points_d20100413'

# id_ = 'd20100413'

# if not os.path.exists('DebateStar/Main Points/' + main_p + '.csv'):
#     os.rename('DebateStar/Main Points/' + main_p, 'DebateStar/Main Points/' + main_p + '.csv')


# sample_main = pd.read_csv('DebateStar/Meta Data/metadata_appended_main_points.csv')
# sample_main = sample_main.loc[sample_main.id == id_, ['For_Main_Points', 'against_Main_Points']]

# for_pts = ast.literal_eval(sample_main.loc[:, 'For_Main_Points'].values[0])
# against_pts = ast.literal_eval(sample_main.loc[:, 'against_Main_Points'].values[0])


In [310]:
# sample_df = pd.read_csv('DebateStar/For Against Scripts/' + df_p)

# for_corpus = sample_df.loc[sample_df.side == 'for', ['script']].values
# for_corpus = [clean_text(c[0]) for c in for_corpus]

# agt_corpus = sample_df.loc[sample_df.side == 'against', ['script']].values
# agt_corpus = [clean_text(c[0]) for c in agt_corpus]


In [312]:
# for_model = models.Word2Vec(for_corpus, workers=3, size=100)
# instance = WmdSimilarity(for_pts, for_model, num_best=10)